In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import librosa
import librosa.display
import os
from scipy.io import wavfile
from scipy.signal import spectrogram
from scipy.stats import kurtosis, skew
import librosa.feature
from scipy import signal
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.metrics import roc_auc_score



In [ ]:
# -----------------
# 1️⃣ Data Preprocessing
# -----------------

def load_labels(labels_csv):
    """ 
    Load labels from CSV file into a dictionary 
    
    """
    labels_df = pd.read_csv(labels_csv)
    labels_dict = dict(zip(labels_df['id'], labels_df['pos_label']))
    return labels_dict

def get_audio_files_labels(folder, labels_dict, target_sr=16000, max_length=3200):
    """ 
    Load all audio file paths and their corresponding labels 
    
    """
    file_paths = []
    labels = []
    
    for file in os.listdir(folder):
        if file.endswith(".wav"):
            file_path = os.path.join(folder, file)
            file_paths.append(file_path)
            labels.append(labels_dict.get(file, 0))  # Default label 0 if not in dict
    
    return file_paths, labels

def load_audio(file_path, target_sr=16000, max_length=3200):
    """ 
    Load and normalize audio file 

    """
    signal, sr = librosa.load(file_path, sr=target_sr)
    signal = librosa.util.fix_length(signal, size=max_length)
    return signal

class AudioDataset(Dataset):
    """
    Custom PyTorch dataset for loading audio files and their labels
    
    """
    def __init__(self, folder, labels_csv, target_sr=16000, max_length=3200):
        self.labels_dict = load_labels(labels_csv)
        self.file_paths, self.labels = get_audio_files_labels(folder, self.labels_dict, target_sr, max_length)
        self.target_sr = target_sr
        self.max_length = max_length
    
    def __len__(self):
        return len(self.file_paths)
    
    def __getitem__(self, idx):
        signal = load_audio(self.file_paths[idx], self.target_sr, self.max_length)
        label = self.labels[idx]
        return torch.tensor(signal, dtype=torch.float32).unsqueeze(0), torch.tensor(label, dtype=torch.float32)

# -----------------
# 2️⃣ Splitting Data into Training, Validation & Test
# -----------------

def create_dataloaders(folder, labels_csv, batch_size=32, train_split=0.7, val_split=0.15, test_split=0.15):
    dataset = AudioDataset(folder, labels_csv)
    total_size = len(dataset)
    train_size = int(train_split * total_size)
    val_size = int(val_split * total_size)
    test_size = total_size - train_size - val_size
    
    train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    
    return train_loader, val_loader, test_loader

# -----------------
# 3️⃣ Model Architecture
# -----------------

class CNN1D(nn.Module):
    def __init__(self):
        super(CNN1D, self).__init__()
        self.conv1 = nn.Conv1d(1, 16, kernel_size=9, stride=1, padding=4)
        self.bn1 = nn.BatchNorm1d(16)
        self.pool1 = nn.MaxPool1d(2)
        
        self.conv2 = nn.Conv1d(16, 32, kernel_size=7, stride=1, padding=3)
        self.bn2 = nn.BatchNorm1d(32)
        self.pool2 = nn.MaxPool1d(2)
        
        self.conv3 = nn.Conv1d(32, 64, kernel_size=5, stride=1, padding=2)
        self.bn3 = nn.BatchNorm1d(64)
        self.pool3 = nn.MaxPool1d(2)
        
        self.fc1 = nn.Linear(64 * 400, 128)  # Adjust based on input size
        self.fc2 = nn.Linear(128, 1)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        x = self.pool1(torch.relu(self.bn1(self.conv1(x))))
        x = self.pool2(torch.relu(self.bn2(self.conv2(x))))
        x = self.pool3(torch.relu(self.bn3(self.conv3(x))))
        x = x.view(x.shape[0], -1)
        x = torch.relu(self.fc1(x))
        x = self.sigmoid(self.fc2(x))
        return x

# -----------------
# 4️⃣ Training Pipeline with Early Stopping
# -----------------

def train_model(train_loader, val_loader, model, criterion, optimizer, num_epochs=10, patience=3):
    """
    Train model with early stopping based on validation loss

    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    
    best_val_loss = float('inf')
    patience_counter = 0
    
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs).squeeze()
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        
        # Validation 
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs).squeeze()
                loss = criterion(outputs, labels)
                val_loss += loss.item()
        
        avg_train_loss = running_loss / len(train_loader)
        avg_val_loss = val_loss / len(val_loader)
        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")
        
        # Early stopping check
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            patience_counter = 0
            torch.save(model.state_dict(), "best_model.pth") 
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print("Early stopping triggered. Training stopped.")
                break
    
    return model

# -----------------
# 5️⃣ Model Evaluation on Test Set
# -----------------

def evaluate_model(test_loader, model):
    """
    Evaluate model on test set
    
    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()
    
    all_labels = []
    all_preds = []
    
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs).squeeze()
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(outputs.cpu().numpy())
    
    roc_auc = roc_auc_score(all_labels, all_preds)
    print(f"Test ROC AUC Score: {roc_auc:.4f}")
    
    return roc_auc


In [ ]:
data_folder = "data/X_train"  
labels_csv = "data/Y_train_ofTdMHi.csv"  

In [ ]:
# Create DataLoaders
train_loader, val_loader, test_loader = create_dataloaders(data_folder, labels_csv, batch_size=32)

# Initialize Model, Loss Function, and Optimizer
model = CNN1D()
criterion = nn.BCELoss()  # Binary Cross Entropy Loss
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train Model
trained_model = train_model(train_loader, val_loader, model, criterion, optimizer, num_epochs=20, patience=5)

# Load the trained model
model = CNN1D()
model.load_state_dict(torch.load("best_model.pth"))



Epoch 1/20, Train Loss: 0.7374, Val Loss: 3.6637
Epoch 2/20, Train Loss: 0.6563, Val Loss: 2.7142
Epoch 3/20, Train Loss: 0.6502, Val Loss: 0.8929
Epoch 4/20, Train Loss: 0.6462, Val Loss: 3.2708
Epoch 5/20, Train Loss: 0.6375, Val Loss: 0.6824
Epoch 6/20, Train Loss: 0.6254, Val Loss: 1.5675
Epoch 7/20, Train Loss: 0.6207, Val Loss: 0.6834
Epoch 8/20, Train Loss: 0.6080, Val Loss: 1.2065
Epoch 9/20, Train Loss: 0.6059, Val Loss: 2.7145
Epoch 10/20, Train Loss: 0.6014, Val Loss: 1.1666
Early stopping triggered. Training stopped.


NameError: name 'roc_auc_score' is not defined

In [ ]:
# Evaluate on the test set
test_roc_auc = evaluate_model(test_loader, model)# Evaluate on the test set


Test ROC AUC Score: 0.5095


In [ ]:
# Define the path for the test data
test_data_folder = "data/X_test"

# Define the same parameters used in training
target_sr = 16000
max_length = 3200

# Load the trained model
model = CNN1D()
model.load_state_dict(torch.load("best_model.pth"))
model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Function to load and preprocess test audio files
def load_test_audio(file_path, target_sr=16000, max_length=3200):
    signal, sr = librosa.load(file_path, sr=target_sr)
    signal = librosa.util.fix_length(signal, size=max_length)  
    return torch.tensor(signal, dtype=torch.float32).unsqueeze(0).to(device)  

# Get test file paths
test_files = [f for f in os.listdir(test_data_folder) if f.endswith(".wav")]
test_file_paths = [os.path.join(test_data_folder, f) for f in test_files]

# Make predictions
predictions = []
with torch.no_grad():
    for file_path in test_file_paths:
        input_tensor = load_test_audio(file_path, target_sr, max_length).unsqueeze(0)  
        prob = model(input_tensor).squeeze().cpu().numpy()  
        predictions.append((os.path.basename(file_path), prob))



In [ ]:
# Convert to DataFrame and save as CSV
submission_df = pd.DataFrame(predictions, columns=["id", "prediction"])
submission_path = "data/X_soumettre_CNN_1D.csv"
submission_df.to_csv(submission_path, index=False)

score pour cette soumission est : 0,5033669433854705

Ce modèle naive n'overfitte pas. Je vais tester d'autres hyperparamètres.